#**Kubernetes homework**

In [3]:
import os
import shutil
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary

In [12]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

--2025-12-03 14:52:59--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-03T15%3A43%3A37Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-03T14%3A43%3A09Z&ske=2025-12-03T15%3A43%3A37Z&sks=b&skv=2018-11-09&sig=3bK6j6Sg1iSMtYkQn7b0XYo%2Fktb%2F%2BENDsovcyIiYhzA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDc3NTM4MCwibmJmIjoxNzY0NzczNTg